<a href="https://colab.research.google.com/github/ShabnaIlmi/Bank-Marketing-Random-Forest-and-Neural-Networks/blob/main/Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer


In [97]:
# Loading the dataset with the proper delimiter (semicolon)
data_full_additional = pd.read_csv('/content/drive/MyDrive/Bank-Marketing-Random-Forest-and-Neural-Networks/bank-additional-full.csv', delimiter=';')


In [98]:
# Displaying the data contained within the dataset
data_full_additional.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# **Data Cleaning**

Removing whitespaces from the object type columns

In [99]:
# Removing whitespaces from the object type column
object_columns = data_full_additional.select_dtypes(include=['object']).columns
data_full_additional[object_columns] = data_full_additional[object_columns].apply(lambda x: x.str.strip())

In [100]:
# Removing unnecessary fullstop(".") from the object type columns
object_columns = data_full_additional.select_dtypes(include=['object']).columns
for col in object_columns:
    data_full_additional[col] = data_full_additional[col].apply(lambda x: x.replace('.', '') if isinstance(x, str) else x)


## **Removing Duplicates**

In [101]:
# Removing duplicates
print(f"Initial dataset shape: {data_full_additional.shape}")
data_full_additional = data_full_additional.drop_duplicates()
print(f"Shape after removing duplicates: {data_full_additional.shape}")

Initial dataset shape: (41188, 21)
Shape after removing duplicates: (41176, 21)


In [102]:
# Display dataset information
print("Dataset Information:")
print(data_full_additional.info())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
Index: 41176 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   job             41176 non-null  object 
 2   marital         41176 non-null  object 
 3   education       41176 non-null  object 
 4   default         41176 non-null  object 
 5   housing         41176 non-null  object 
 6   loan            41176 non-null  object 
 7   contact         41176 non-null  object 
 8   month           41176 non-null  object 
 9   day_of_week     41176 non-null  object 
 10  duration        41176 non-null  int64  
 11  campaign        41176 non-null  int64  
 12  pdays           41176 non-null  int64  
 13  previous        41176 non-null  int64  
 14  poutcome        41176 non-null  object 
 15  emp.var.rate    41176 non-null  float64
 16  cons.price.idx  41176 non-null  float64
 17  cons.conf.idx  

## **Handling Missing Data**

In [103]:
# Handling "unknown" values in categorical features
categorical_cols = [
    'job', 'marital', 'education', 'default', 'housing',
    'loan', 'contact', 'month', 'day_of_week', 'poutcome'
]

In [104]:
# Creating an empty list
unknown_values = []

# Checking for 'unknown' values in the data of categorical columns
for col in categorical_cols:
    unknown_count = (data_full_additional[col] == 'unknown').sum()
    if unknown_count > 0:
        unknown_values.append({'Column': col, " Count": unknown_count})

# Converting the list into a pandas DataFrame
unknown_values_df = pd.DataFrame(unknown_values)

# Displaying the DataFrame
print("\nColumns with 'unknown' values and their counts:")
print(unknown_values_df)


Columns with 'unknown' values and their counts:
      Column   Count
0        job     330
1    marital      80
2  education    1730
3    default    8596
4    housing     990
5       loan     990


In [105]:
# Displaying the unique values and their counts relevant to each column containing "unknown" values
for _, row in unknown_values_df.iterrows():
    col = row['Column']
    unique_values = data_full_additional[col].unique()
    value_counts = data_full_additional[col].value_counts()
    print(value_counts)
    print(" ")

job
admin            10419
blue-collar       9253
technician        6739
services          3967
management        2924
retired           1718
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64
 
marital
married     24921
single      11564
divorced     4611
unknown        80
Name: count, dtype: int64
 
education
universitydegree      12164
highschool             9512
basic9y                6045
professionalcourse     5240
basic4y                4176
basic6y                2291
unknown                1730
illiterate               18
Name: count, dtype: int64
 
default
no         32577
unknown     8596
yes            3
Name: count, dtype: int64
 
housing
yes        21571
no         18615
unknown      990
Name: count, dtype: int64
 
loan
no         33938
yes         6248
unknown      990
Name: count, dtype: int64
 


In [106]:
# List of categorical columns to handle specifically with mode
columns_to_handle_with_mode = ['marital', 'default', 'housing', 'loan']

# Replacing "unknown" with NaN for imputation
data_full_additional[columns_to_handle_with_mode] = data_full_additional[columns_to_handle_with_mode].replace('unknown', np.nan)

In [107]:
# Display dataset information
print("Dataset Information:")
print(data_full_additional.info())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
Index: 41176 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   job             41176 non-null  object 
 2   marital         41096 non-null  object 
 3   education       41176 non-null  object 
 4   default         32580 non-null  object 
 5   housing         40186 non-null  object 
 6   loan            40186 non-null  object 
 7   contact         41176 non-null  object 
 8   month           41176 non-null  object 
 9   day_of_week     41176 non-null  object 
 10  duration        41176 non-null  int64  
 11  campaign        41176 non-null  int64  
 12  pdays           41176 non-null  int64  
 13  previous        41176 non-null  int64  
 14  poutcome        41176 non-null  object 
 15  emp.var.rate    41176 non-null  float64
 16  cons.price.idx  41176 non-null  float64
 17  cons.conf.idx  

Replacing the missing ("unknown") data contained within the "default" and the "loan" column with the mode ("no" and "no") since the mode represents more than 90% of those columns this is considered as the most optimal choice to handled those missing values. Additionally, missing values in the columns, martial and housing also replaced with the mode since more than 60% of those columns also consists of the mode and those columns has comparably less number of missing values.

In [108]:
# Replacing NaN with the mode in the specified columns
for col in columns_to_handle_with_mode:
    mode_value = data_full_additional[col].mode()[0]
    data_full_additional[col] = data_full_additional[col].fillna(mode_value)

In [109]:
# Verifying the changes and checking for the columns which contain 'unknown' values
print("\nColumns with 'unknown' values and their counts:\n")
for _, row in unknown_values_df.drop_duplicates(subset='Column').iterrows():
    col = row['Column']
    # Count the number of "unknown" values in the column
    unknown_count = data_full_additional[col].str.contains('unknown', case=False, na=False).sum()
    if unknown_count > 0:
        print(f"{col} : {unknown_count}")


Columns with 'unknown' values and their counts:

job : 330
education : 1730


Considering the missing values in the columns, "job" and "education" as a seperate category during the encoding process

In [ ]:
#  One-hot encoding for non-ordinal categorical variables
non_ordinal_cols = ['job', 'marital', 'contact', 'poutcome', 'month', 'day_of_week']
data_full_additional = pd.get_dummies(data_full_additional, columns=non_ordinal_cols, drop_first=True)

In [ ]:
# Mapping ordinal categorical variables
education_mapping = {
    'basic.4y': 1, 'basic.6y': 2, 'basic.9y': 3, 'high.school': 4,
    'illiterate': 5, 'professional.course': 6, 'university.degree': 7
}
default_mapping = {'no': 0, 'yes': 1}
housing_mapping = {'no': 0, 'yes': 1}
loan_mapping = {'no': 0, 'yes': 1}

data_full_additional['education'] = data_full_additional['education'].map(education_mapping)
data_full_additional['default'] = data_full_additional['default'].map(default_mapping)
data_full_additional['housing'] = data_full_additional['housing'].map(housing_mapping)
data_full_additional['loan'] = data_full_additional['loan'].map(loan_mapping)

In [ ]:
# Excluding the 'duration' feature for realistic predictive modeling
data_full_additional = data_full_additional.drop(columns=['duration'])

In [ ]:
# Scaling numerical features
numerical_cols = [
    'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
    'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'
]
scaler = StandardScaler()
data_full_additional[numerical_cols] = scaler.fit_transform(data_full_additional[numerical_cols])

In [ ]:
# Target encoding
# Encode the target variable (y)
data_full_additional['y'] = data_full_additional['y'].map({'yes': 1, 'no': 0})

In [ ]:
# Adding derived features from numerical data
data_full_additional['interaction_emp_cons'] = data_full_additional['emp.var.rate'] * data_full_additional['cons.price.idx']
data_full_additional['interaction_nr_employed_conf'] = data_full_additional['nr.employed'] * data_full_additional['cons.conf.idx']

In [ ]:
# Spliting features and target variable
X_full_additional = data_full_additional.drop(columns=['y'])
y_full_additional = data_full_additional['y']


In [ ]:
# Splitting the data into Train-Test
X_train, X_test, y_train, y_test = train_test_split(
    X_full_additional, y_full_additional, test_size=0.2, random_state=42
)

In [ ]:
# Save processed data for model training if needed
X_train.to_csv('/content/drive/MyDrive/banking_system/processed_X_train.csv', index=False)
y_train.to_csv('/content/drive/MyDrive/banking_system/processed_y_train.csv', index=False)
X_test.to_csv('/content/drive/MyDrive/banking_system/processed_X_test.csv', index=False)
y_test.to_csv('/content/drive/MyDrive/banking_system/processed_y_test.csv', index=False)

In [ ]:
# Displaying dataset shapes
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")